In [ ]:
from google.colab import drive
drive.mount('/content/drive')

<img src="https://raw.githubusercontent.com/LotusZaheer/BN_CL/master/Banner.jpg" align="center"/>

<h4>Universidad Industrial de Santander</h4>
<h3>Inteligencia Artificial I - J1</h3>
<h3>Profesor Gustavo Adolfo Garzón Villamizar</h3>
<br>
Andrés Felipe Uribe García - 2160793
<br>
Julian Orlando Rodríguez Villamizar - 2170137

Los datos provienen del dataset https://www.kaggle.com/chetankv/dogs-cats-images el cual fue utilizado para un problema de clasificación binaria de perros y gatos


Para el preprocesado del dataset de gatos se utilizó un for en bash con la instrución 
<br>
convert i.jpg -colorspace Gray i.jpg
<br>
La cual transforma todas las imagenes de color a escala de grises utilizando la aplicación ImageMagick, la cual tambien ofrece otro tipo de transformaciones
<br>
Tutorial rapido: https://blog.desdelinux.net/como-manipular-imagenes-desde-el-terminal/
Proyecto: https://github.com/ImageMagick/ImageMagick


In [ ]:
import os
import imageio
import numpy as np
from glob import glob
import tensorflow as tf
from random import randint
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.display import Image #Error de plt
from tensorflow.keras.layers import *
from urllib.request import urlretrieve
from IPython.display import clear_output
from keras.models import Sequential, Model

In [ ]:
IMG_SIZE = 128 #A 256 demora una eternidad

#Ruta Raiz
PATH = "/content/drive/My Drive/BN_CL/"

#Datos de entrada a Blanco y negro
INPATH = PATH + "train/"#https://drive.google.com/drive/folders/1NkVi1_hya6J7DRfH6zq6xi_7iUraAL_N?usp=sharing

#Datos de Generados a Color
OUTPATH = PATH + "target/" #https://drive.google.com/drive/folders/1IiIjvRJakrMCB4RDRyXYCq20Zew35sGy?usp=sharing

#Datos de Esperados a Color
PREDPATH = PATH + "output/"

#Cargamos SOLO el nombre de las imagenes en un listado
imgurls = !ls -1 "{INPATH}"

In [ ]:
n=1000
#Tamaño del test del 80%
train_n = round(n*0.8)

#Listado randomizado
randurls = np.copy(imgurls)

np.random.seed(23)
np.random.shuffle(randurls)

#Partición de test y train
tr_urls = randurls[:train_n]
ts_urls = randurls[train_n:]

print( "Tamaño del dataset :", len(imgurls))
print( "Tamaño de los datos de entrenamiento :", len(tr_urls))
print( "Tamaño de los datos de testeo :", len(ts_urls))

In [ ]:
#Para reducir el costo computacional
@tf.function
def rescalado(inimg, tgimg, height, width):
  inimg = tf.image.resize(inimg, [height, width])
  tgimg = tf.image.resize(tgimg, [height, width])

  return inimg, tgimg

In [ ]:
#En lugar de trabajar con valores de 0 a 255
#usamos datos entre -1 y 1
def normalizado(inimg, tgimg):
  #Tanh trabaja de -1 a 1 por lo cual para aprovechar su dominio no trabajaremos de 0 a 1 
  #si no de -1 a 1
  #Ver en capa resultado ↓↓↓↓↓↓↓↓
  inimg = (inimg /127.5) -1
  tgimg = (tgimg /127.5) -1
  return inimg, tgimg

In [ ]:
#Cargamos las imagenes
#Nombre del archivo 
@tf.function
def load_image(nombre):

  #No lo comprendo en su totalidad :U
  #####Buscar documentacion#####
  inimg = tf.cast(tf.image.decode_jpeg(tf.io.read_file(INPATH + nombre)), tf.float32)[..., :3]
  tgimg = tf.cast(tf.image.decode_jpeg(tf.io.read_file(OUTPATH + nombre)), tf.float32)[..., :3]
  
  #Cargamos las imagenes e inmediatamente las rescalamos
  #El mismo tamaño para altura y ancho
  inimg, tgimg = rescalado(inimg, tgimg, IMG_SIZE, IMG_SIZE)

  #Normalizamos
  inimg, tgimg = normalizado(inimg, tgimg) 

  return inimg, tgimg

In [ ]:
#Probamos la función load_image
#Visualizamos las imagenes para ver el material a trabajar
#randurls[imagen])[BW=0;Color=1])
#Una imagen aletoria cada vez
rimg = randint(0,n)
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.imshow( ((load_image(randurls[rimg])[0])+1)/2 )
plt.subplot(1,2,2)
plt.imshow( ((load_image(randurls[rimg])[1])+1)/2 )
plt.show()
#En numeriños :u
#(load_image(randurls[rimg])[1])

In [ ]:
# :v/ Al fin el dataset como tal \v:

#Cargamos el listado de direcciones
X_train = tf.data.Dataset.from_tensor_slices(tr_urls)
X_test = tf.data.Dataset.from_tensor_slices(ts_urls)

#Cargamos las imagenes con nuestra funcion
#¡IMPORTANTE! Revisar el numero de hilos o verás el azul dominar tu pantalla :u
X_train = X_train.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE) #Mejor dejarlo en automatico
X_test = X_test.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

#Distribuimos los datos por lotes
X_train = X_train.batch(1)
X_test = X_test.batch(1)


#Probamos que las imagenes se hayan cargado en el dataset
#Visualizamos las imagenes
for inimg, tgimg in X_test.take(2):
  plt.figure(figsize=(16,6))
  plt.subplot(1,2,1)
  plt.imshow(((inimg[0,...])+1)/2) #En caso de ignorar los lotes -> (((tgimg)+1)/2)
  plt.subplot(1,2,2)
  plt.imshow(((tgimg[0,...])+1)/2)
  plt.show()
#rof

In [ ]:
#Downsampling
def Compresion(filtros, batch_norm=True):
  
  #Capas, como una cebolla
  C = Sequential()
  #Ruido Gausiano de media 0 y desviación estandar de 0.02 
  initializer = tf.random_normal_initializer(0, 0.02)

  #Capa convolucional
  C.add(Conv2D(filtros,
               kernel_size=4,
               strides=2, #Vamos reduciendo el tamaño de imagen a la mitad
               padding="same",
               kernel_initializer=initializer,
               use_bias=not batch_norm))
  
  #Capa de Batch
  if batch_norm:
    C.add(BatchNormalization())
  #Capa de activación
  C.add(ReLU())
  return C

In [ ]:
#Upsampling
def Expansion(filtros, dropout=True):
  
  #Capas, como una cebolla
  E = Sequential()
  #Ruido Aleatorio
  initializer = tf.random_normal_initializer(0, 0.02)

  #Capa convolucional
  E.add(Conv2DTranspose(filtros,
                        kernel_size=4,
                        strides=2, #Vamos reduciendo el tamaño de imagen a la mitad
                        padding="same",
                        kernel_initializer=initializer,
                        use_bias=False))

  #Capa de Batch para trabajar por lotes
  if dropout:
    E.add(Dropout(0.5))

  #Capa de activación
  E.add(ReLU())

  return E

In [ ]:
def Generador():
  
  inputs = Input(shape=[None, None, 3]) #Img_size no especificados, 3 colores

  Encoder = [         #Tamaño
      Compresion(64, batch_norm=False), #128
      Compresion(128),#64
      Compresion(256),#32
      Compresion(256),#16
      Compresion(256),#8
      Compresion(256),#4
      Compresion(256),#2
      ##Para trabajar con 256 descomente esta linea
      #Compresion(256),#1
  ]
  
  Decoder = [        #Tamaño
      ##Para trabajar con 256 descomente esta linea
      #Expansion(256),#2
      Expansion(256),#4
      Expansion(256),#8
      Expansion(256, dropout=False),#16
      Expansion(256, dropout=False),#32
      Expansion(128, dropout=False),#64
      Expansion(64, dropout=False), #128
  ]


  I = tf.random_normal_initializer(0, 0.02)
  
  #Tamaño 128
  Resultado = Conv2DTranspose(filters=3,
                              kernel_size=4,
                              strides=2,
                              padding="same",
                              kernel_initializer=I, 
                              activation='tanh') #Errores con la normalización
  
  x = inputs
  s = [] #Lista con los saltos de conexión 
  concat = Concatenate()

  #Pasamos el resultado de cada capa a la siguiente
  for i in Encoder:
    x = i(x)
    s.append(x)
  #rof

  #La primera en entrar debe ser la ultima en salir
  #El elemento de la capa de tamaño 1 al ser el centro la eliminamos
  s = reversed(s[:-1]) 
  
  
  for j, sl in zip(Decoder, s):
    x = j(x)
    x = concat([x, sl]) #Concexion de los datos que saltaron
  #rof

  output = Resultado(x)

  return Model(inputs=inputs, outputs=output) 
 

In [ ]:
#Utilizaremos un patchGAN discriminator :u
#En lugar de Evaluar por imagenes lo haremos por parches
#aprovechando que tenemos el codigo por piezas

def Discriminador():
  ini = Input(shape=[None, None, 3], name="input_img")
  gen = Input(shape=[None, None, 3], name="gen_img")

  con = concatenate([ini, gen])

  I = tf.random_normal_initializer(0, 0.02)
  
  dec1 = Compresion(32, batch_norm=False)(con)
  dec2 = Compresion(64)(dec1)
  dec3 = Compresion(128)(dec2)
  dec4 = Compresion(256)(dec3)
  
  Resultado = Conv2DTranspose(filters=1,
                              kernel_size=4,
                              strides=1,
                              kernel_initializer=I, 
                              padding="same")(dec4)
  
  return Model(inputs=[ini, gen], outputs=Resultado)

In [ ]:
#Creamos el generador
G = Generador()
#Resumen del Modelo Generador
G.summary()

In [ ]:
#Creamos el Discriminador
D = Discriminador()
#Resumen del Modelo Discriminador
D.summary()

In [ ]:
#Test de ejemplo:
#Lo que genera el modelo
og = G(((inimg+1)*255), training=False) 

#Test de ejemplo:
#Lo que considera es falso
od = D([((inimg+1)*255), og], training=False)

#PLoteamos para ver las diferencias
plt.figure(figsize=(16,6))

plt.subplot(1,2,1)
plt.imshow(og[0,...])

plt.subplot(1,2,2)
plt.imshow(od[0,...,-1],
           vmin=-5, vmax=5,
           cmap='RdBu_r') #'RdBu_r'
plt.colorbar()
plt.show()

In [ ]:
#Entropía cruzada de cada pixel obtenido
#'from_logits' con ella normalizamos para qeu exista en [0. .1]
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
#Evaluamos el Discriminador

#Con ella vamos a tomar los datos del generador y el discriminador 
#para que así vayan mejorando uno a la par del otro
def discrimator_loss(real_output, pred_output):
  
  #Diferencia entre la imagen real y lo que el discriminador piensa es real
  #Con ones_like establecemos que la imagen es real, dando por buena toda su extensión 
  real_loss = loss_object(tf.ones_like(real_output), real_output)
  
  #Diferencia entre la imagen predicha y lo que el discriminador piensa es real
  #Con zero_like establecemos que la imagen es predicha, dando por falsa toda su extensión 
  pred_loss = loss_object(tf.zeros_like(pred_output), pred_output)
  
  #Sumamos las diferencias de ambas operaciones
  total_loss = real_loss + pred_loss

  return total_loss

In [ ]:
#Evaluamos el Generador
L = 100 

#El mapa generado por el discriminador, Imagen generada y la imagen esperada
def generator_loss(output_disc, pred_output, target):

  #Diferencia entre la imagen predicha y lo que el discriminador piensa es real
  #Con ones_like establecemos que la imagen es predicha es adecuada dandola por verdadera
  gan_loss = loss_object(tf.ones_like(output_disc), output_disc)
  
  #Error absoluto
  #La diferencia entre lo que generamos y lo esperabamos
  abs_loss = tf.reduce_mean(tf.abs(target - pred_output))

  #Multiplicando por Lambda damos un mayor valor al error absoluto de la predicción
  total_loss =  gan_loss +(L * abs_loss)
  
  return total_loss

In [ ]:
#Optimizamos utilizando Adam con los valores
generador_optimizer = tf.keras.optimizers.Adam()
discriminador_optimizer = tf.keras.optimizers.Adam()

In [ ]:
#Para visualizar en cada etapa

def generate_images(model, ts_input, tar, save_filename=False, display_imgs=True):
  
  prediction = model(ts_input, training=True)

  if save_filename:
    tf.keras.preprocessing.image.save_img(PATH + 'output/' + save_filename + '.jpg', prediction[0,...])
    
  plt.figure(figsize=(16,6))
  
  plt.subplot(1, 3, 1)
  plt.title('Entrada')
  plt.imshow( ((ts_input[0])+1)/2 )
  plt.axis('off')
  
  plt.subplot(1, 3, 2)
  plt.title('Ground Truth')
  plt.imshow( ((tar[0])+1)/2 )
  plt.axis('off')
  
  plt.subplot(1, 3, 3)
  plt.title('Predición')
  plt.imshow( ((prediction[0])+1)/2 )
  plt.axis('off')
  plt.show()

In [ ]:
#Entrenamos el modelo
#Pasamos la imagen de entrada y la imagen esperada
@tf.function
def train_step(generar, discriminar, input_image, target):
  
  with tf.GradientTape() as gen_tape, tf.GradientTape() as discr_tape:
    
    #Imagen de salida a partir de la de entrada
    output_image = generar(input_image, training=True)

    #Pasamos la imagen generada y la imagen de entrada
    output_pred_discr = discriminar([output_image, input_image], training=True)

    #Imagen esperada y la imagen de entrada
    output_target_discr = discriminar([target, input_image], training=True)

    #Recibimos la evaluación del discriminador
    discr_loss = discrimator_loss(output_target_discr, output_pred_discr)
    #Recibimos la evaluación del generador
    gen_loss = generator_loss(output_pred_discr,output_image, target)

    #Calculamos el descenso del gradiente del generador
    generator_grads = gen_tape.gradient(gen_loss, generar.trainable_variables)
    #Calculamos el descenso del gradiente del discriminador
    discriminator_grads = discr_tape.gradient(discr_loss, discriminar.trainable_variables)

    generador_optimizer.apply_gradients(zip(generator_grads, generar.trainable_variables))
    discriminador_optimizer.apply_gradients(zip(discriminator_grads, discriminar.trainable_variables))

In [ ]:
def train(generar, discriminar, train, test, epochs):
  for epoch in range(epochs):

    imgi = 0
    for input_image, target in train:
      print ('epoch ' + str(epoch) + ' -> Entrenando -> ' + str(imgi) + '/' + str(len(tr_urls)))
      imgi = imgi + 1
      train_step(generar, discriminar, input_image, target)
      clear_output(wait=True) 
    #rof 

    for inp, tar in test.take(5):
      generate_images(generar, inp, tar, #str(imgi) + '_' + 
                      str(epoch), display_imgs=True)
    #rof
  #rof


In [ ]:
###Guardar avances
checkpoint_prefix = os.path.join(PATH + "check", "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generador_optimizer,
                                discriminator_optimizer=discriminador_optimizer,
                                generator=G,
                                discriminator=D)

In [ ]:
#Cargamos el avance anterior
checkpoint.restore(tf.train.latest_checkpoint(PATH + "check"))
#Entrenamiento
train(G, D, X_train, X_test, 100)
#Guardamos el avance actual
checkpoint.save(file_prefix=checkpoint_prefix)

In [ ]:
#Cargamos SOLO el nombre de las imagenes en un listado
pred = !ls -1 "{PREDPATH}"

now = datetime.now()

#Guardamos el avance del día en forma de un lindo gif :3
filenames = sorted(pred)
imgs = []
for f in filenames:
    imgs.append(imageio.imread(PREDPATH + f))
imageio.mimsave(PATH + 'Gifs/' + str(now) + '.gif', imgs, duration=0.1, subrectangles=True)

In [ ]:
#####################################################################
#####Borra el check anterior y los outputs para ahorrar espacio######
#####################################################################